# 🚀 TP Final - Ciencia de Datos
## Entrenamiento de Modelo de Lenguaje para Generación de Reseñas

Este notebook entrena un modelo de lenguaje para generar reseñas de productos de Amazon.

### 📋 Objetivos:
- Cargar y preparar datos de reseñas de Amazon
- Entrenar un modelo de lenguaje para generar reseñas
- Evaluar la calidad de las reseñas generadas
- Analizar la coherencia entre ratings y sentimientos

## 📦 Importaciones

In [ ]:
# Importaciones necesarias
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling, pipeline
import torch
import re

# Para traducción
try:
    from googletrans import Translator
    GOOGLE_TRANS_AVAILABLE = True
except ImportError:
    GOOGLE_TRANS_AVAILABLE = False
    print("⚠️ googletrans no disponible. Instalar con: pip install googletrans==4.0.0rc1")

try:
    from transformers import MarianMTModel, MarianTokenizer
    MARIAN_AVAILABLE = True
except ImportError:
    MARIAN_AVAILABLE = False
    print("⚠️ MarianMT no disponible. Instalar con: pip install sentencepiece")

print("✅ Librerías importadas correctamente")

## 🔍 Verificación de Hardware

In [ ]:
# Verificar GPU
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("Usando CPU - optimizando para mejor rendimiento")
    # Optimizaciones para CPU
    torch.set_num_threads(8)  # Usar más núcleos de CPU
    print(f"Núcleos CPU utilizados: {torch.get_num_threads()}")

## 📊 Carga y Preparación de Datos

In [ ]:
# Leer dataset
print("📖 Cargando dataset...")
df = pd.read_csv('Amazon_Unlocked_Mobile.csv')
df = df.dropna(subset=['Reviews', 'Rating'])  # eliminamos reseñas y ratings vacíos
df = df[['Product Name', 'Brand Name', 'Price', 'Rating', 'Reviews']]

# Convertir rating a entero
df['Rating'] = df['Rating'].astype(int)

print(f"📊 Dataset cargado: {len(df)} registros")
print(f"📈 Distribución de ratings:")
print(df['Rating'].value_counts().sort_index())

In [ ]:
# Balancear el dataset: igual cantidad de reseñas por rating
max_per_rating = 800  # Aumentado para mejor aprendizaje
balanced_df = pd.concat([
    df[df['Rating'] == rating].sample(n=min(max_per_rating, len(df[df['Rating'] == rating])), random_state=42)
    for rating in range(1, 6)
])

print(f"⚖️ Dataset balanceado: {len(balanced_df)} registros")
print(f"📊 Distribución por rating:")
for rating in range(1, 6):
    count = len(balanced_df[balanced_df['Rating'] == rating])
    print(f"  {rating} estrellas: {count} reseñas")

## 🧹 Funciones de Limpieza y Procesamiento

In [ ]:
def limpiar_texto(texto):
    """Limpia el texto de caracteres especiales y normaliza espacios"""
    if pd.isna(texto):
        return ""
    
    texto = str(texto)
    # Remover caracteres especiales pero mantener puntuación básica
    texto = re.sub(r'[^\w\s\.\,\!\?\-\']', '', texto)
    # Normalizar espacios
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def crear_prompt_mejorado(row):
    """Crea un prompt más estructurado y específico"""
    rating = row['Rating']
    producto = limpiar_texto(row['Product Name'] or 'Producto')
    marca = limpiar_texto(row['Brand Name'] or 'Marca')
    precio = str(row['Price'])
    reseña = limpiar_texto(row['Reviews'])
    
    # Determinar sentimiento esperado basado en rating
    if rating == 1:
        sentimiento = "muy negativo"
        instruccion = "Escribe una reseña muy crítica y negativa"
    elif rating == 2:
        sentimiento = "negativo"
        instruccion = "Escribe una reseña negativa con algunos aspectos positivos"
    elif rating == 3:
        sentimiento = "neutral"
        instruccion = "Escribe una reseña equilibrada con pros y contras"
    elif rating == 4:
        sentimiento = "positivo"
        instruccion = "Escribe una reseña positiva con algunas críticas menores"
    else:  # rating == 5
        sentimiento = "muy positivo"
        instruccion = "Escribe una reseña muy positiva y entusiasta"
    
    # Prompt mejorado con instrucciones claras
    prompt = f"""INSTRUCCIÓN: {instruccion} para este producto.

PRODUCTO: {producto}
MARCA: {marca}
PRECIO: {precio}
RATING: {rating} estrellas ({sentimiento})

RESEÑA: {reseña}

FIN"""

    return prompt

print("✅ Funciones de procesamiento definidas")

## 🎯 Creación de Prompts Estructurados

In [ ]:
# Aplicar el nuevo prompt
balanced_df['text'] = balanced_df.apply(crear_prompt_mejorado, axis=1)

print('📝 Ejemplo de texto mejorado:')
print(balanced_df['text'].iloc[0])
print('Tipo:', type(balanced_df['text'].iloc[0]))

In [ ]:
# Crear dataset de HuggingFace
dataset = Dataset.from_pandas(balanced_df[['text']])

# Dividir dataset en entrenamiento y validación
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset['train']
eval_dataset = dataset['test']

print(f"📚 Dataset de entrenamiento: {len(train_dataset)} ejemplos")
print(f"🔍 Dataset de validación: {len(eval_dataset)} ejemplos")

## 🔠 Tokenización

In [ ]:
# Configurar tokenizador con modelo más grande
model_name = "gpt2"  # Cambiado a gpt2 completo para mejor capacidad
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # Aumentamos la longitud máxima a 256 tokens para capturar mejor el contexto
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256,  # Aumentado significativamente
    )

print('🔤 Tokenizador configurado')
print('📝 Ejemplo de tokenización:')
print(tokenize_function({"text": [balanced_df['text'].iloc[0]]}))

In [ ]:
# Tokenizar datasets
print("🔄 Tokenizando datasets...")

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_train = tokenized_train.remove_columns(['text'])

tokenized_eval = eval_dataset.map(tokenize_function, batched=True)
tokenized_eval = tokenized_eval.remove_columns(['text'])

print("✅ Tokenización completada")

## 🚀 Configuración del Entrenamiento

In [ ]:
# Cargar modelo
model = AutoModelForCausalLM.from_pretrained(model_name)
print(f"🤖 Modelo {model_name} cargado")

In [ ]:
# Configurar argumentos de entrenamiento optimizados
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,  # Reducido para evitar overfitting
    per_device_train_batch_size=4,  # Reducido para el modelo más grande
    save_steps=200,  # Guardar más frecuentemente
    save_total_limit=3,  # Mantener 3 checkpoints
    logging_steps=50,  # Logging más frecuente para monitoreo
    prediction_loss_only=True,
    remove_unused_columns=False,
    # Optimizaciones para aprendizaje óptimo
    dataloader_num_workers=2,  # Workers moderados para Windows
    gradient_accumulation_steps=8,  # Acumular gradientes para batch efectivo de 32
    warmup_steps=100,  # Warmup apropiado
    learning_rate=1e-4,  # Learning rate más alto para mejor aprendizaje
    weight_decay=0.01,  # Regularización
    # Optimizaciones adicionales
    fp16=False,  # Desactivar para CPU
    bf16=False,  # Desactivar para CPU
    optim="adamw_torch",  # Optimizador eficiente
    lr_scheduler_type="cosine",  # Scheduler óptimo
    max_grad_norm=1.0,  # Gradient clipping
    evaluation_strategy="steps",  # Evaluar durante el entrenamiento
    eval_steps=200,  # Evaluar cada 200 pasos
)

print("⚙️ Configuración de entrenamiento optimizada")

In [ ]:
# Configurar data collator y trainer
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("🎯 Trainer configurado y listo para entrenar")

## 🏋️ Entrenamiento del Modelo

In [ ]:
# Iniciar entrenamiento
print("🚀 Iniciando entrenamiento...")
print("⏱️ Esto puede tomar varios minutos...")

trainer.train()

print("✅ Entrenamiento completado")

## 💾 Guardado del Modelo

In [ ]:
# Guardar modelo y tokenizer
print("💾 Guardando modelo y tokenizer...")

model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

print("✅ Modelo guardado en './results'")

## 🎭 Funciones de Generación Mejoradas

In [ ]:
def crear_prompt_generacion(producto, marca, precio, rating):
    """Crea un prompt específico para generación"""
    if rating == 1:
        sentimiento = "muy negativo"
        instruccion = "Escribe una reseña muy crítica y negativa"
    elif rating == 2:
        sentimiento = "negativo"
        instruccion = "Escribe una reseña negativa con algunos aspectos positivos"
    elif rating == 3:
        sentimiento = "neutral"
        instruccion = "Escribe una reseña equilibrada con pros y contras"
    elif rating == 4:
        sentimiento = "positivo"
        instruccion = "Escribe una reseña positiva con algunas críticas menores"
    else:  # rating == 5
        sentimiento = "muy positivo"
        instruccion = "Escribe una reseña muy positiva y entusiasta"
    
    prompt = f"""INSTRUCCIÓN: {instruccion} para este producto.

PRODUCTO: {producto}
MARCA: {marca}
PRECIO: {precio}
RATING: {rating} estrellas ({sentimiento})

RESEÑA:"""
    
    return prompt

def generar_resena_mejorada(producto, marca, precio, rating):
    """Genera una reseña mejorada con mejor control"""

    prompt = crear_prompt_generacion(producto, marca, precio, rating)

    # Configurar el generador con parámetros optimizados
    generator = pipeline(
        'text-generation',
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=80,  # Generar hasta 80 tokens nuevos
        do_sample=True,
        temperature=0.7,  # Temperatura moderada para balance entre creatividad y coherencia
        top_p=0.85,  # Nucleus sampling
        top_k=40,  # Top-k sampling
        repetition_penalty=1.3,  # Penalizar repeticiones
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # Configuración adicional
        num_beams=1,  # Sin beam search para más creatividad
        length_penalty=1.0,
        no_repeat_ngram_size=3,  # Evitar repetición de n-gramas
    )

    # Generar texto
    resultado = generator(prompt, num_return_sequences=1)[0]['generated_text']

    # Extraer solo la reseña generada
    if "RESEÑA:" in resultado:
        reseña = resultado.split("RESEÑA:")[1].split("FIN")[0].strip()
    else:
        reseña = resultado[len(prompt):].strip()

    # Limpiar la reseña
    reseña = re.sub(r'\s+', ' ', reseña).strip()
    
    # Si la reseña es muy corta, intentar generar más
    if len(reseña.split()) < 10:
        # Continuar la generación
        prompt_continuacion = resultado + " "
        resultado2 = generator(prompt_continuacion, num_return_sequences=1)[0]['generated_text']
        parte2 = resultado2[len(prompt_continuacion):].strip()
        reseña = reseña + " " + parte2
        reseña = re.sub(r'\s+', ' ', reseña).strip()

    return reseña

def analizar_sentimiento(reseña):
    """Analiza el sentimiento de una reseña"""
    reseña_lower = reseña.lower()
    
    # Palabras positivas y negativas más específicas
    palabras_positivas = [
        'good', 'great', 'excellent', 'love', 'like', 'amazing', 'perfect', 'wonderful',
        'fantastic', 'awesome', 'outstanding', 'superb', 'brilliant', 'fabulous',
        'satisfied', 'happy', 'pleased', 'impressed', 'recommend', 'best', 'quality'
    ]
    
    palabras_negativas = [
        'bad', 'terrible', 'awful', 'hate', 'dislike', 'horrible', 'worst', 'disappointed',
        'poor', 'cheap', 'broken', 'defective', 'useless', 'waste', 'regret', 'avoid',
        'problem', 'issue', 'faulty', 'unreliable', 'slow', 'expensive', 'overpriced'
    ]
    
    positivas = sum(1 for palabra in palabras_positivas if palabra in reseña_lower)
    negativas = sum(1 for palabra in palabras_negativas if palabra in reseña_lower)
    
    return positivas, negativas

print("✅ Funciones de generación definidas")

## 🎯 Prueba de Generación de Reseñas

In [ ]:
# Probar generaciones mejoradas
print("🎯 Generando reseñas de ejemplo con el modelo mejorado...\n")

# Ejemplos de productos
productos = [
    ("iPhone 15 Pro", "Apple", "$999", 1),
    ("Samsung Galaxy S24", "Samsung", "$799", 2),
    ("Google Pixel 8", "Google", "$699", 3),
    ("OnePlus 12", "OnePlus", "$599", 4),
    ("Xiaomi 14", "Xiaomi", "$499", 5)
]

for producto, marca, precio, rating in productos:
    print(f"📱 {producto} ({marca}) - {precio} - {rating}⭐")
    reseña = generar_resena_mejorada(producto, marca, precio, rating)
    print(f"📝 Reseña: {reseña}")
    print(f"📊 Longitud: {len(reseña.split())} palabras")

    # Analizar sentimiento
    positivas, negativas = analizar_sentimiento(reseña)
    print(f"😊 Palabras positivas: {positivas}")
    print(f"😞 Palabras negativas: {negativas}")

    # Verificar si el sentimiento coincide con el rating
    if rating >= 4 and positivas > negativas:
        print("✅ Sentimiento COINCIDE con rating alto")
    elif rating <= 2 and negativas > positivas:
        print("✅ Sentimiento COINCIDE con rating bajo")
    elif rating == 3 and abs(positivas - negativas) <= 2:
        print("✅ Sentimiento COINCIDE con rating neutral")
    else:
        print("❌ Sentimiento NO coincide con rating")

    print("-" * 80)

## 📊 Análisis de Resultados

In [ ]:
# Mostrar métricas de entrenamiento de forma segura
print("Métricas de entrenamiento:")

if hasattr(trainer, 'state'):
    print(f"Pasos totales: {trainer.state.global_step}")

    # Mostrar todas las métricas disponibles
    if trainer.state.log_history:
        print("📊 Historial de métricas:")
        for i, log in enumerate(trainer.state.log_history[-5:]):  # Últimos 5 logs
            print(f"  Paso {log.get('step', 'N/A')}:")
            for key, value in log.items():
                if key != 'step':
                    print(f"    {key}: {value}")
    else:
        print("No hay historial de métricas disponible")
else:
    print("No se encontró información del estado del trainer")

print("\n🎉 ¡Entrenamiento completado exitosamente!")

## 🔧 Funciones Adicionales de Traducción

In [ ]:
def traducir_con_google(texto, idioma_destino='es'):
    """Traduce usando Google Translate"""
    if not GOOGLE_TRANS_AVAILABLE:
        return "Traducción no disponible - instalar googletrans"

    try:
        translator = Translator()
        traduccion = translator.translate(texto, dest=idioma_destino)
        return traduccion.text
    except Exception as e:
        return f"Error en traducción: {e}"

def traducir_con_marian(texto, idioma_origen='en', idioma_destino='es'):
    """Traduce usando MarianMT (más preciso)"""
    if not MARIAN_AVAILABLE:
        return "Traducción no disponible - instalar sentencepiece"

    try:
        # Modelo específico para inglés a español
        model_name = f'Helsinki-NLP/opus-mt-{idioma_origen}-{idioma_destino}'
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)

        # Tokenizar y traducir
        inputs = tokenizer(texto, return_tensors="pt", padding=True)
        translated = model.generate(**inputs)
        traduccion = tokenizer.decode(translated[0], skip_special_tokens=True)

        return traduccion
    except Exception as e:
        return f"Error en traducción: {e}"

def traducir_resena(resena, metodo='marian', idioma_destino='es'):
    """Traduce una reseña usando el método especificado"""

    if metodo == 'google':
        return traducir_con_google(resena, idioma_destino)
    elif metodo == 'marian':
        return traducir_con_marian(resena, 'en', idioma_destino)
    else:
        return "Método de traducción no válido"

print("✅ Funciones de traducción definidas")

## 🌍 Prueba de Traducción (Opcional)

In [ ]:
# Ejemplo de traducción de una reseña generada
print("🌍 Probando traducción de reseñas...\n")

# Generar una reseña de ejemplo
reseña_ejemplo = generar_resena_mejorada("iPhone 15 Pro", "Apple", "$999", 5)
print(f"📝 Reseña original: {reseña_ejemplo}")

# Traducir usando MarianMT
if MARIAN_AVAILABLE:
    traduccion = traducir_resena(reseña_ejemplo, metodo='marian')
    print(f"🌍 Traducción (MarianMT): {traduccion}")
else:
    print("⚠️ MarianMT no disponible para traducción")

# Traducir usando Google Translate
if GOOGLE_TRANS_AVAILABLE:
    traduccion_google = traducir_resena(reseña_ejemplo, metodo='google')
    print(f"🌍 Traducción (Google): {traduccion_google}")
else:
    print("⚠️ Google Translate no disponible para traducción")